In [1]:
import numpy as np
from pandas import read_csv

In [2]:
#lstm_data = np.genfromtxt('./sample_data/lstm.csv', delimiter=',', skip_header=True)
lstm_data = read_csv('lstm.csv')
lstm_data = lstm_data.drop(['Date'], axis=1)
print(lstm_data)

Empty DataFrame
Columns: [High, Low, Open, Close, Volume, Adj Close, Polarity Score]
Index: []


In [3]:
lstm_data_X = lstm_data.drop(['Close'], axis=1)
lstm_data_y = lstm_data['Close']

In [4]:
look_back = 7 # how many days to look back
batch_size = 5 # size of batches used when training
n_feat = 8 # number of features 
n_target = 1
n_train = 60
n_validation = 5
n_test = 6

In [5]:
lstm_data_X_train = lstm_data_X.iloc[:60,:]
lstm_data_X_val = lstm_data_X.iloc[60:65,:]
lstm_data_X_test = lstm_data_X.iloc[60:71,:]

lstm_data_y_train = lstm_data_y.iloc[:60]
lstm_data_y_val = lstm_data_y.iloc[60:65]
lstm_data_y_test = lstm_data_y.iloc[60:71]
# Convert to numpy arrays
X_train = lstm_data_X_train.to_numpy()
X_val = lstm_data_X_val.to_numpy()
X_test = lstm_data_X_test.to_numpy()
y_train = lstm_data_y_train.to_numpy()
y_val = lstm_data_y_val.to_numpy()
y_test = lstm_data_y_test.to_numpy()

In [6]:
X_test

array([], shape=(0, 6), dtype=object)

In [7]:
y_test

array([], dtype=object)

In [8]:
import keras
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, ThresholdedReLU, MaxPooling2D, Embedding, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

In [9]:
train_data_gen = TimeseriesGenerator(X_train, y_train, length=look_back, batch_size=batch_size)
val_data_gen = TimeseriesGenerator(X_val, y_val, length=look_back, batch_size=batch_size)
test_data_gen = TimeseriesGenerator(X_test, y_test, length=look_back, batch_size=batch_size)

ValueError: `start_index+length=7 > end_index=-1` is disallowed, as no part of the sequence would be left to be used as current step.

In [ ]:
# check generator dimensions
for i in range(len(train_data_gen)):
    x, y = train_data_gen[i]
    print(x.shape, y.shape)

In [ ]:
model_lstm = Sequential()
model_lstm.add(LSTM(32, input_shape=(look_back, n_feat), return_sequences=True))
model_lstm.add(Dropout(0.1))
model_lstm.add(LSTM(32))
model_lstm.add(Dropout(0.1))
model_lstm.add(Dense(1))
model_lstm.compile(optimizer="adam", loss='mse', metrics=["mse"])
print(model_lstm.summary())

In [ ]:
hist = model_lstm.fit_generator(train_data_gen,
                                        steps_per_epoch=10,
                                        epochs=20,
                                        verbose=1,
                                        validation_data=val_data_gen)

In [ ]:
pred = model_lstm.predict(test_data_gen)

In [ ]:
test_data_gen[0]

In [ ]:
pred